Experiment 1: Effect of time step, l

In [30]:
import numpy as np
import math

using the implicit method

In [37]:
# Given constants
P = 2000
mu = 1/32
gamma1 = 0.1
gamma2 = 0.05
tau = 0.4
B1 = 0.06
B2 = 0.5
c = 4
X0 = 8000
Y01 = 200
Y02 = 300
delta_t_values = [0.01, 1, 3, 3.3, 10, 1000]

# Function to perform one iteration of the system of equations using Gauss-Seidel-like method
def iterate_system(Xn, Y1n, Y2n, dt):
    Xnp1 = (Xn + P * dt) / (1 + dt * (mu + c / (Xn + Y1n + Y2n) * (B1 * Y1n + B2 * Y2n)))
    Y1np1 = Y1n / (1 + dt * (mu + gamma1 + tau - (B1 * c * Xnp1) / (Xnp1 + Y1n + Y2n)))
    Y2np1 = Y2n / (1 + dt * (mu + gamma2 + tau - (B2 * c * Xnp1) / (Xnp1 + Y1np1 + Y2n)))
    return Xnp1, Y1np1, Y2np1

# Function to solve the system for all time steps
def solve_system(delta_t):
    Xn = X0
    Y1n = Y01
    Y2n = Y02
    max_iterations = 200000
    tol = 1e-10
    
    # Gauss-Seidel-like iteration
    for _ in range(max_iterations):
        Xnp1, Y1np1, Y2np1 = iterate_system(Xn, Y1n, Y2n, delta_t)
        
        # Check convergence
        if abs(Xnp1 - Xn) < tol and abs(Y1np1 - Y1n) < tol and abs(Y2np1 - Y2n) < tol:
            return Xnp1, Y1np1, Y2np1
        
        # Update variables for next iteration
        Xn, Y1n, Y2n = Xnp1, Y1np1, Y2np1
    
    # If max_iterations reached without convergence
    raise ValueError("Failed to converge within the maximum number of iterations.")

# Solve the system for all time steps
for dt in delta_t_values:
    X, Y1, Y2 = solve_system(dt)
    print(f"For delta_t = {dt}: X = {X}, Y1 = {Y1}, Y2 = {Y2}")


For delta_t = 0.01: X = 1290.3225806456214, Y1 = 8.873416070380664e-09, Y2 = 4072.056975302486
For delta_t = 1: X = 1290.3225806451783, Y1 = 1.459474974662082e-15, Y2 = 4072.0569752826727
For delta_t = 3: X = 63999.99999999903, Y1 = 5.655120406472925e-95, Y2 = 2.8507236165568116e-193
For delta_t = 3.3: X = 63999.99999999915, Y1 = 3.3149975955781596e-93, Y2 = 1.0467850118779407e-193
For delta_t = 10: X = 63999.999999999745, Y1 = 1.0274511853837072e-70, Y2 = 9.125316420824659e-139
For delta_t = 1000: X = 64000.0, Y1 = 5.043036460757328e-28, Y2 = 2.0509825879325442e-36


RK4 method is shown below

In [1]:
def dX_dt(P, mu, B1, B2, c, X, Y1, Y2):
    N = X + Y1 + Y2
    if N == 0:
        return 0  
    else:
        return P - mu * X - (1/N) * B1 * c * X * Y1 - (1/N) * B2 * c * X * Y2


def dY1_dt(B1, c, mu, gamma1, tau, X, Y1):
    N = X + Y1 + Y2
    if N == 0:
        return 0  
    else:
        return (1/N) * B1 * c * X * Y1 - (mu + gamma1 + tau) * Y1



def dY2_dt(B2, c, mu, gamma2, tau, X, Y2):
    N = X + Y1 + Y2
    if N == 0:
        return 0  
    else:
        return (1/N) * B2 * c * X * Y2 - (mu + gamma2 + tau) * Y2

def RK4_step(P, mu, B1, B2, c, gamma1, gamma2, tau, X, Y1, Y2, dt):
    k1_X = dX_dt(P, mu, B1, B2, c, X, Y1, Y2)
    k1_Y1 = dY1_dt(B1, c, mu, gamma1, tau, X, Y1)
    k1_Y2 = dY2_dt(B2, c, mu, gamma2, tau, X, Y2)

    X_half = X + 0.5 * dt * k1_X
    Y1_half = Y1 + 0.5 * dt * k1_Y1
    Y2_half = Y2 + 0.5 * dt * k1_Y2

    k2_X = dX_dt(P, mu, B1, B2, c, X_half, Y1_half, Y2_half)
    k2_Y1 = dY1_dt(B1, c, mu, gamma1, tau, X_half, Y1_half)
    k2_Y2 = dY2_dt(B2, c, mu, gamma2, tau, X_half, Y2_half)

    X_half2 = X + 0.5 * dt * k2_X
    Y1_half2 = Y1 + 0.5 * dt * k2_Y1
    Y2_half2 = Y2 + 0.5 * dt * k2_Y2

    k3_X = dX_dt(P, mu, B1, B2, c, X_half2, Y1_half2, Y2_half2)
    k3_Y1 = dY1_dt(B1, c, mu, gamma1, tau, X_half2, Y1_half2)
    k3_Y2 = dY2_dt(B2, c, mu, gamma2, tau, X_half2, Y2_half2)

    X_end = X + dt * k3_X
    Y1_end = Y1 + dt * k3_Y1
    Y2_end = Y2 + dt * k3_Y2

    return X_end, Y1_end, Y2_end

# Parameters and initial conditions
P = 2000
mu = 1/32
gamma1 = 0.1
gamma2 = 0.05
tau = 0.4
B1 = 0.06
B2 = 0.5
c = 4
X0 = 8000
Y01 = 200
Y02 = 300
dt_values = [0.01, 1, 3, 3.3, 10, 1000]

# Simulation
for dt in dt_values:
    X = X0
    Y1 = Y01
    Y2 = Y02
    print(f"Time step: {dt}")
    for i in range(130):  # Adjust number of steps as needed
        X_prev, Y1_prev, Y2_prev = X, Y1, Y2
        X, Y1, Y2 = RK4_step(P, mu, B1, B2, c, gamma1, gamma2, tau, X, Y1, Y2, 1/dt)
    # Check convergence using epsilon
    epsilon = 1e-6  # Define a small tolerance
    if abs(X - X_prev) < epsilon and abs(Y1 - Y1_prev) < epsilon and abs(Y2 - Y2_prev) < epsilon:
        print("Converged")
    else:
        print("Diverged")
    print()



Time step: 0.01
Converged

Time step: 1
Converged

Time step: 3
Converged

Time step: 3.3
Diverged

Time step: 10
Diverged

Time step: 1000
Diverged

